# Divide DFs for both versions, keeping visitor and visit ids, changing 'gender' column values.

In [1]:
import pandas as pd

In [2]:
# import reusable functions from utils directory
import sys
sys.path.append('../../utils')
import functions

In [3]:
combined_cleaned_df = pd.read_csv('../../data/clean/combined_cleaned_data.csv')
combined_cleaned_df.head()

,client_id,visitor_id,visit_id,process_step,date_time
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04


In [4]:
merged_demo_experiment_df = pd.read_csv('./final_demo_df_clean.csv')
merged_demo_experiment_df

,client_id,client_tenure_years,client_tenure_months,client_age,gender,num_accounts,balance,calls_last_6_months,logons_last_6_months
0,836976,6,73,60,Unspecified,2,45105.30,6,9
1,2304905,7,94,58,Unspecified,2,110860.30,6,9
2,1439522,5,64,32,Unspecified,2,52467.79,6,9
3,1562045,16,198,49,Male,2,67454.65,3,6
4,5126305,12,145,33,Female,2,103671.75,0,3
...,...,...,...,...,...,...,...,...,...
70589,7993686,4,56,38,Unspecified,3,1411062.68,5,5
70590,8981690,12,148,31,Male,2,101867.07,6,6
70591,333913,16,198,61,Female,2,40745.00,3,3
70592,1573142,21,255,68,Male,3,475114.69,4,4


In [5]:
merged_client_df = pd.merge(combined_cleaned_df, merged_demo_experiment_df, on='client_id', how='left')

In [6]:

merged_client_df.rename(columns={'process_step' : 'step', 'client_tenure_years': 'tenure_years', 'client_tenure_months': 'tenure_months', 'client_age' : 'age', 'num_accounts' : 'accounts'}, inplace=True)
functions.floats_to_ints(merged_client_df, 'tenure_years')
functions.floats_to_ints(merged_client_df, 'tenure_months')
functions.floats_to_ints(merged_client_df, 'age')
functions.floats_to_ints(merged_client_df, 'accounts')
functions.floats_to_ints(merged_client_df, 'calls_last_6_months')
functions.floats_to_ints(merged_client_df, 'logons_last_6_months')



merged_client_df

,client_id,visitor_id,visit_id,step,date_time,tenure_years,tenure_months,age,gender,accounts,balance,calls_last_6_months,logons_last_6_months
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07,5,64,79,Unspecified,2,189023.86,1,4
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51,5,64,79,Unspecified,2,189023.86,1,4
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22,5,64,79,Unspecified,2,189023.86,1,4
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13,5,64,79,Unspecified,2,189023.86,1,4
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04,5,64,79,Unspecified,2,189023.86,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
744636,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:46:10,<NA>,<NA>,<NA>,NaN,<NA>,NaN,<NA>,<NA>
744637,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:45:29,<NA>,<NA>,<NA>,NaN,<NA>,NaN,<NA>,<NA>
744638,9668240,388766751_9038881013,922267647_3096648104_968866,step_1,2017-05-24 18:44:51,<NA>,<NA>,<NA>,NaN,<NA>,NaN,<NA>,<NA>
744639,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:44:34,<NA>,<NA>,<NA>,NaN,<NA>,NaN,<NA>,<NA>


In [7]:
replacement_dict_steps = {
    'start' : 0,
    'step_1' : 1,
    'step_2' : 2,
    'step_3' : 3,
    'confirm' : 4
}

merged_client_df['step'] = merged_client_df['step'].map(replacement_dict_steps)

merged_client_df


,client_id,visitor_id,visit_id,step,date_time,tenure_years,tenure_months,age,gender,accounts,balance,calls_last_6_months,logons_last_6_months
0,9988021,580560515_7732621733,781255054_21935453173_531117,3,2017-04-17 15:27:07,5,64,79,Unspecified,2,189023.86,1,4
1,9988021,580560515_7732621733,781255054_21935453173_531117,2,2017-04-17 15:26:51,5,64,79,Unspecified,2,189023.86,1,4
2,9988021,580560515_7732621733,781255054_21935453173_531117,3,2017-04-17 15:19:22,5,64,79,Unspecified,2,189023.86,1,4
3,9988021,580560515_7732621733,781255054_21935453173_531117,2,2017-04-17 15:19:13,5,64,79,Unspecified,2,189023.86,1,4
4,9988021,580560515_7732621733,781255054_21935453173_531117,3,2017-04-17 15:18:04,5,64,79,Unspecified,2,189023.86,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
744636,9668240,388766751_9038881013,922267647_3096648104_968866,0,2017-05-24 18:46:10,<NA>,<NA>,<NA>,NaN,<NA>,NaN,<NA>,<NA>
744637,9668240,388766751_9038881013,922267647_3096648104_968866,0,2017-05-24 18:45:29,<NA>,<NA>,<NA>,NaN,<NA>,NaN,<NA>,<NA>
744638,9668240,388766751_9038881013,922267647_3096648104_968866,1,2017-05-24 18:44:51,<NA>,<NA>,<NA>,NaN,<NA>,NaN,<NA>,<NA>
744639,9668240,388766751_9038881013,922267647_3096648104_968866,0,2017-05-24 18:44:34,<NA>,<NA>,<NA>,NaN,<NA>,NaN,<NA>,<NA>


# Add Error Rate Column

In [11]:
# Detect backward steps (errors) for all clients
merged_client_df['backward_step'] = merged_client_df.groupby('client_id')['step'].diff() < 0

# Count the number of unique clients who experienced at least one error
clients_with_errors = merged_client_df[merged_client_df['backward_step']]['client_id'].nunique()

# Count the total number of errors for each client
errors_per_client = merged_client_df[merged_client_df['backward_step']].groupby('client_id').size()

# Total number of errors across all clients
total_errors = merged_client_df['backward_step'].sum()

print(f'Number of clients with at least one error: {clients_with_errors}')
print('Errors per client:')
print(errors_per_client)
print(f'Total number of errors across all clients: {total_errors}')

Number of clients with at least one error: 105082
Errors per client:
client_id
169        4
546        4
555        4
647        4
722        6
          ..
9999729    7
9999768    7
9999832    1
9999839    4
9999875    4
Length: 105082, dtype: int64
Total number of errors across all clients: 446362


In [12]:
# add column 'error_count'
merged_client_df['error_count'] = merged_client_df['client_id'].map(errors_per_client).fillna(0).astype(int)

In [13]:
merged_client_df

,client_id,visitor_id,visit_id,step,date_time,tenure_years,tenure_months,age,gender,accounts,balance,calls_last_6_months,logons_last_6_months,backward_step,error_count
0,9988021,580560515_7732621733,781255054_21935453173_531117,3,2017-04-17 15:27:07,5,64,79,Unspecified,2,189023.86,1,4,False,5
1,9988021,580560515_7732621733,781255054_21935453173_531117,2,2017-04-17 15:26:51,5,64,79,Unspecified,2,189023.86,1,4,True,5
2,9988021,580560515_7732621733,781255054_21935453173_531117,3,2017-04-17 15:19:22,5,64,79,Unspecified,2,189023.86,1,4,False,5
3,9988021,580560515_7732621733,781255054_21935453173_531117,2,2017-04-17 15:19:13,5,64,79,Unspecified,2,189023.86,1,4,True,5
4,9988021,580560515_7732621733,781255054_21935453173_531117,3,2017-04-17 15:18:04,5,64,79,Unspecified,2,189023.86,1,4,False,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
744636,9668240,388766751_9038881013,922267647_3096648104_968866,0,2017-05-24 18:46:10,<NA>,<NA>,<NA>,NaN,<NA>,NaN,<NA>,<NA>,True,12
744637,9668240,388766751_9038881013,922267647_3096648104_968866,0,2017-05-24 18:45:29,<NA>,<NA>,<NA>,NaN,<NA>,NaN,<NA>,<NA>,False,12
744638,9668240,388766751_9038881013,922267647_3096648104_968866,1,2017-05-24 18:44:51,<NA>,<NA>,<NA>,NaN,<NA>,NaN,<NA>,<NA>,False,12
744639,9668240,388766751_9038881013,922267647_3096648104_968866,0,2017-05-24 18:44:34,<NA>,<NA>,<NA>,NaN,<NA>,NaN,<NA>,<NA>,True,12
